In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import xgboost
import cv2
import imblearn

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [4]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
np.sum(df.isnull())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
X=df.drop('Outcome', axis=1)

In [8]:
Y=df['Outcome']

In [9]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [10]:
Y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [11]:
X.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000


In [12]:
X.corr()[(X.corr()>=0.8) | (X.corr()<=-0.8)]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Glucose,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
BloodPressure,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
SkinThickness,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Insulin,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
BMI,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
DiabetesPedigreeFunction,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
Age,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


# OVERSAMPLING

In [13]:
from imblearn.over_sampling import RandomOverSampler

ros=RandomOverSampler()
x_res,y_res=ros.fit_sample(X,Y)

In [14]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_res,y_res, random_state=42, test_size=0.2, stratify=y_res)

In [15]:
print(len(x_train),len(x_test), end=' ')
print()

print(len(y_train),len(y_test),end=' ')

800 200 
800 200 

In [16]:
for x in set(X):
    print('{}....{}'.format(x,len(X[X==x])))

Glucose....768
SkinThickness....768
BloodPressure....768
BMI....768
DiabetesPedigreeFunction....768
Insulin....768
Age....768
Pregnancies....768


In [17]:
for y in set(Y):
    print('{}....{}'.format(y,len(Y[Y==y])))

0....500
1....268


In [18]:
for x in set(x_res):
    print('{}....{}'.format(x,len(x_res[x_res==x])))

Glucose....1000
SkinThickness....1000
BloodPressure....1000
BMI....1000
DiabetesPedigreeFunction....1000
Insulin....1000
Age....1000
Pregnancies....1000


In [19]:
for y in set(y_res):
    print('{}....{}'.format(y,len(y_res[y_res==y])))

0....500
1....500


In [20]:
for x in set(x_train):
    print('{}....{}'.format(x,len(x_train[x_train==x])))

Glucose....800
SkinThickness....800
BloodPressure....800
BMI....800
DiabetesPedigreeFunction....800
Insulin....800
Age....800
Pregnancies....800


In [21]:
for x in set(x_test):
    print('{}....{}'.format(x,len(x_test[x_test==x])))

Glucose....200
SkinThickness....200
BloodPressure....200
BMI....200
DiabetesPedigreeFunction....200
Insulin....200
Age....200
Pregnancies....200


In [22]:
for y in set(y_train):
    print('{}....{}'.format(y,len(y_train[y_train==y])))

0....400
1....400


In [23]:
for y in set(y_test):
    print('{}....{}'.format(y,len(y_test[y_test==y])))

0....100
1....100


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [25]:
#lr=LogisticRegression()
#lr.fit(x_train, y_train)

from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train_scaled=sc_x.fit_transform(x_train)


#svc=SVC(kernel='rbf', gamma=2, C=0.000000001)
svc=SVC()

In [26]:
#from sklearn.model_selection import cross_val_predict

#y_probas= cross_val_predict(lr,x_train, y_train,cv=5,
#                              method='predict_proba')

In [27]:
#precisions, recalls, thresholds= precision_recall_curve(y_train, y_probas[:,1])

In [28]:
#px.line(x=recalls,y=precisions,labels=dict(x="Recall", y="Precision"))

In [29]:
#from sklearn.metrics import roc_curve

#fpr, tpr, thresholds=roc_curve(y_train, y_probas[:,1])

In [30]:
#plt.plot(fpr, tpr, "b:", label="Linear reg")
#plt.plot(fpr, tpr, linewidth=2, label='linear reg')
#plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
#plt.legend(loc="lower right")
#plt.show()

In [31]:
#from sklearn.metrics import roc_auc_score
#roc_auc_score(y_train, y_probas[:,1])

In [32]:
#from sklearn.metrics import precision_recall_curve

#precisions, recalls, thresholds= precision_recall_curve(y_train, y_scores[:,1])

In [33]:
#plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
#plt.plot(thresholds, recalls[:-1], "g-", label="Recall")

In [34]:
from sklearn.model_selection import GridSearchCV

#kernel='rbf', gamma=2, C=0.000000001

params={
    'kernel':['rbf'],
    'gamma':[1,2,3],
    'C':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1,2,3,4,5,6,7,8,9,10,100]
}

#scoring={'precision':'precision', 'recall':'recall'}
#gs=GridSearchCV(svc, cv=5, param_grid=params, n_jobs=-1, 
#                scoring=scoring,
#               refit='precision')

gs=GridSearchCV(svc, cv=10, param_grid=params, n_jobs=-1, 
                scoring='f1')
gs.fit(x_train_scaled, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07,
                               1e-08, 1e-09, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                               100],
                         'gamma': [1, 2, 3], 'kernel': ['rbf']},
             scoring='f1')

In [35]:
#gamma=2, C=0.000000001

In [36]:
print(gs.best_params_)
print(gs.best_score_)

{'C': 2, 'gamma': 1, 'kernel': 'rbf'}
0.8145102099868387


In [37]:
svc_best=SVC(C= float(gs.best_params_.get('C')), gamma= float(gs.best_params_.get('gamma')), 
             kernel= gs.best_params_.get('kernel'))

svc_best.fit(x_train_scaled, y_train)

print(float(gs.best_params_.get('C')))
print(float(gs.best_params_.get('gamma')))
print(str(gs.best_params_.get('kernel')))

2.0
1.0
rbf


In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score


y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

[[398   2]
 [  4 396]]


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       400
           1       0.99      0.99      0.99       400

    accuracy                           0.99       800
   macro avg       0.99      0.99      0.99       800
weighted avg       0.99      0.99      0.99       800



Precision = 0.9949748743718593
Recall = 0.99
f1-score = 0.9924812030075189


In [39]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import cross_val_predict


#y_train_pred=svc_best.predict(x_train_scaled)
y_train_pred= cross_val_predict(svc_best, x_train_scaled, y_train,cv=10)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

[[299 101]
 [ 55 345]]


              precision    recall  f1-score   support

           0       0.84      0.75      0.79       400
           1       0.77      0.86      0.82       400

    accuracy                           0.81       800
   macro avg       0.81      0.81      0.80       800
weighted avg       0.81      0.81      0.80       800



Precision = 0.773542600896861
Recall = 0.8625
f1-score = 0.8156028368794327


In [40]:
y_test_pred=svc_best.predict(sc_x.transform(x_test))
print(confusion_matrix(y_test, y_test_pred))
print()
print()
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))
print()
print('Precision =',precision_score(y_test, y_test_pred))
print('Recall =',recall_score(y_test, y_test_pred))
print('f1-score =',f1_score(y_test, y_test_pred))

[[71 29]
 [ 7 93]]


              precision    recall  f1-score   support

           0       0.91      0.71      0.80       100
           1       0.76      0.93      0.84       100

    accuracy                           0.82       200
   macro avg       0.84      0.82      0.82       200
weighted avg       0.84      0.82      0.82       200


Precision = 0.7622950819672131
Recall = 0.93
f1-score = 0.8378378378378378


# SMOTE

In [41]:
from sklearn.linear_model import LogisticRegression

In [42]:
from imblearn.combine import SMOTETomek

smt=SMOTETomek(random_state=42)
x_res,y_res= smt.fit_sample(X,Y)

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_res,y_res, random_state=42, test_size=0.2, stratify=y_res)

In [44]:
print(len(x_train),len(x_test), end=' ')
print()

print(len(y_train),len(y_test),end=' ')

761 191 
761 191 

In [45]:
for x in set(X):
    print('{}....{}'.format(x,len(X[X==x])))

Glucose....768
SkinThickness....768
BloodPressure....768
BMI....768
DiabetesPedigreeFunction....768
Insulin....768
Age....768
Pregnancies....768


In [46]:
for y in set(Y):
    print('{}....{}'.format(y,len(Y[Y==y])))

0....500
1....268


In [47]:
for x in set(x_res):
    print('{}....{}'.format(x,len(x_res[x_res==x])))

Glucose....952
SkinThickness....952
BloodPressure....952
BMI....952
DiabetesPedigreeFunction....952
Insulin....952
Age....952
Pregnancies....952


In [48]:
for y in set(y_res):
    print('{}....{}'.format(y,len(y_res[y_res==y])))

0....476
1....476


In [49]:
for x in set(x_train):
    print('{}....{}'.format(x,len(x_train[x_train==x])))

Glucose....761
SkinThickness....761
BloodPressure....761
BMI....761
DiabetesPedigreeFunction....761
Insulin....761
Age....761
Pregnancies....761


In [50]:
for x in set(x_test):
    print('{}....{}'.format(x,len(x_test[x_test==x])))

Glucose....191
SkinThickness....191
BloodPressure....191
BMI....191
DiabetesPedigreeFunction....191
Insulin....191
Age....191
Pregnancies....191


In [51]:
for y in set(y_train):
    print('{}....{}'.format(y,len(y_train[y_train==y])))

0....380
1....381


In [52]:
for y in set(y_test):
    print('{}....{}'.format(y,len(y_test[y_test==y])))

0....96
1....95


In [53]:
#lr=LogisticRegression()
#lr.fit(x_train, y_train)
from sklearn.preprocessing import StandardScaler
sc_x=StandardScaler()
x_train_scaled=sc_x.fit_transform(x_train)


#svc=SVC(kernel='rbf', gamma=1, C=0.001)
svc=SVC()

In [54]:
from sklearn.model_selection import GridSearchCV

#kernel='rbf', gamma=2, C=0.000000001

params={
    'kernel':['rbf'],
    'gamma':[1,2,3],
    'C':[1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1,2,3,4,5,6,7,8,9,10,100]
}

#scoring={'precision':'precision', 'recall':'recall'}
#gs=GridSearchCV(svc, cv=5, param_grid=params, n_jobs=-1, 
#                scoring=scoring,
#               refit='precision')


gs=GridSearchCV(svc, cv=10, param_grid=params, n_jobs=-1, 
                scoring='f1')

gs.fit(x_train_scaled, y_train)

GridSearchCV(cv=10, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07,
                               1e-08, 1e-09, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                               100],
                         'gamma': [1, 2, 3], 'kernel': ['rbf']},
             scoring='f1')

In [55]:
#gamma=1, C=0.001

In [56]:
print(gs.best_params_)
print(gs.best_score_)

{'C': 1, 'gamma': 1, 'kernel': 'rbf'}
0.8334538610679493


In [57]:
svc_best=SVC(C= float(gs.best_params_.get('C')), gamma= float(gs.best_params_.get('gamma')), 
             kernel= str(gs.best_params_.get('kernel')))
svc_best.fit(x_train_scaled, y_train)

print(float(gs.best_params_.get('C')))
print(float(gs.best_params_.get('gamma')))
print(str(gs.best_params_.get('kernel')))

1.0
1.0
rbf


In [58]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score

y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

[[368  12]
 [  5 376]]


              precision    recall  f1-score   support

           0       0.99      0.97      0.98       380
           1       0.97      0.99      0.98       381

    accuracy                           0.98       761
   macro avg       0.98      0.98      0.98       761
weighted avg       0.98      0.98      0.98       761



Precision = 0.9690721649484536
Recall = 0.9868766404199475
f1-score = 0.977893368010403


In [59]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import cross_val_predict

y_train_pred= cross_val_predict(svc_best, x_train_scaled, y_train,cv=10)
#y_train_pred=svc_best.predict(x_train_scaled)
print(confusion_matrix(y_train,y_train_pred))
print()
print()
print(classification_report(y_train,y_train_pred))
print()
print()
print('Precision =',precision_score(y_train, y_train_pred))
print('Recall =',recall_score(y_train, y_train_pred))
print('f1-score =',f1_score(y_train, y_train_pred))

[[294  86]
 [ 47 334]]


              precision    recall  f1-score   support

           0       0.86      0.77      0.82       380
           1       0.80      0.88      0.83       381

    accuracy                           0.83       761
   macro avg       0.83      0.83      0.82       761
weighted avg       0.83      0.83      0.82       761



Precision = 0.7952380952380952
Recall = 0.8766404199475065
f1-score = 0.8339575530586766


In [60]:
y_test_pred=svc_best.predict(sc_x.transform(x_test))
print(confusion_matrix(y_test, y_test_pred))
print()
print()
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred))
print()
print('Precision =',precision_score(y_test, y_test_pred))
print('Recall =',recall_score(y_test, y_test_pred))
print('f1-score =',f1_score(y_test, y_test_pred))

[[71 25]
 [11 84]]


              precision    recall  f1-score   support

           0       0.87      0.74      0.80        96
           1       0.77      0.88      0.82        95

    accuracy                           0.81       191
   macro avg       0.82      0.81      0.81       191
weighted avg       0.82      0.81      0.81       191


Precision = 0.7706422018348624
Recall = 0.8842105263157894
f1-score = 0.823529411764706


In [61]:
cvres= gs.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(mean_score), params)
#cvres

0.8593556439467251 {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
0.8255670132605241 {'C': 0.1, 'gamma': 2, 'kernel': 'rbf'}
0.8191262063537487 {'C': 0.1, 'gamma': 3, 'kernel': 'rbf'}
0.820303112429596 {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'}
0.8234075540095561 {'C': 0.01, 'gamma': 2, 'kernel': 'rbf'}
0.8191262063537487 {'C': 0.01, 'gamma': 3, 'kernel': 'rbf'}
0.820303112429596 {'C': 0.001, 'gamma': 1, 'kernel': 'rbf'}
0.8234075540095561 {'C': 0.001, 'gamma': 2, 'kernel': 'rbf'}
0.8191262063537487 {'C': 0.001, 'gamma': 3, 'kernel': 'rbf'}
0.820303112429596 {'C': 0.0001, 'gamma': 1, 'kernel': 'rbf'}
0.8234075540095561 {'C': 0.0001, 'gamma': 2, 'kernel': 'rbf'}
0.8191262063537487 {'C': 0.0001, 'gamma': 3, 'kernel': 'rbf'}
0.820303112429596 {'C': 1e-05, 'gamma': 1, 'kernel': 'rbf'}
0.8234075540095561 {'C': 1e-05, 'gamma': 2, 'kernel': 'rbf'}
0.8191262063537487 {'C': 1e-05, 'gamma': 3, 'kernel': 'rbf'}
0.820303112429596 {'C': 1e-06, 'gamma': 1, 'kernel': 'rbf'}
0.8234075540095561 {'C': 1e-06, 'ga